## Próba implementacji metod zastosowanych przez dr. Kontka w artykule 

["**Weryfikacja wyniku drugiej tury wyborów prezydenckich w Polsce w 2025 roku: Przeliczenie głosów z użyciem przestrzennie grupowanej metody MAD**"](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=5296441)

ale **przy zastosowaniu całkowicie losowych danych i losowego grupowania komisji wyborczych**

### Założenie i hipoteza
Celem eksperymentu jest sprawdzenie, czy wykrywalność tzw. anomalii wyborczych (wg metody MAD) zależy faktycznie od przestrzennego podobieństwa komisji, czy też wynika jedynie z naturalnego rozrzutu procentowego poparcia — nawet przy idealnie losowych i uczciwych danych.

W ramach testu:

* Wygenerowano dane dla ponad 31 000 komisji wyborczych,

* Podzielono je losowo na około 2200 grup (10–16 komisji każda),

* W każdej grupie rozkład głosów oparto na symetrycznym rozkładzie normalnym wokół 50%,

* Dodano drobne różnice między 1. a 2. turą (na wzór naturalnej dynamiki kampanii),

Zarówno wartości, jak i przypisanie do grup są losowe – z zachowaniem statystycznego realizmu.

### Interpretacja
Jeśli metoda dr. Kontka działa zgodnie z założeniem, że komisje w pobliżu geograficznym mają zbliżone wyniki, to losowe grupowanie komisji (czyli całkowite oderwanie od kontekstu przestrzennego) powinno znacząco ograniczyć liczbę wykrywanych anomalii.

Jeśli jednak mimo pełnej losowości danych i grupowania — przy założeniu poprawnych, symetrycznych rozkładów — liczba anomalii pozostaje podobna jak w danych rzeczywistych, może to oznaczać, że metoda MAD nie wykrywa realnych odstępstw od "normalności", ale flaguje naturalne fluktuacje jako podejrzane — nawet tam, gdzie żadnej anomalii nie ma.

W takim przypadku pojawia się uzasadniona wątpliwość, czy metoda nadaje się do wykrywania fałszerstw wyborczych — zwłaszcza jeśli generuje porównywalne poziomy alarmów nawet w czysto symulowanych, losowych i uczciwych danych.

In [6]:
import pandas as pd
import numpy as np

from utilities import presidential_data, clustering, methods

### generate fake data

In [37]:
import numpy as np
import pandas as pd

# 🔧 Parametry
n_commissions = 31627
n_groups = 2208
votes_per = 1000

# 📊 Rzeczywiste wyniki 1. tury
pct_trz_r1 = 30.8
pct_naw_r1 = 29.1
pct_other_r1 = 100 - pct_trz_r1 - pct_naw_r1  # = 40.1%

# Odchylenia standardowe
std_r1 = 1.5
std_r2 = 2.0

rng = np.random.default_rng(123)
group_sizes = rng.integers(10, 17, size=n_groups)
group_sizes[-1] += n_commissions - group_sizes.sum()

data = []
polling_id, teryt_base, postal_base = 100000, 140000, 30000

for g, size in enumerate(group_sizes):
    for _ in range(size):
        # --- 1. tura ---
        trz_r1_pct = rng.normal(pct_trz_r1, std_r1)
        naw_r1_pct = rng.normal(pct_naw_r1, std_r1)
        other_r1_pct = 100 - trz_r1_pct - naw_r1_pct

        # Obróbka progowa i normalizacja
        arr = np.array([trz_r1_pct, naw_r1_pct, other_r1_pct])
        arr = np.clip(arr, 0, None)
        arr = arr / arr.sum()

        trz_r1 = int(round(votes_per * arr[0]))
        naw_r1 = int(round(votes_per * arr[1]))

        # --- 2. tura (symetrycznie wokół 50%) ---
        trz_r2_pct = rng.normal(50, std_r2)
        trz_r2_pct = np.clip(trz_r2_pct, 0, 100)
        trz_r2 = int(round(votes_per * trz_r2_pct / 100))
        naw_r2 = votes_per - trz_r2

        data.append({
            "teryt_gmina": f"{teryt_base + g}",
            "polling_station_id": polling_id,
            "postal_code": f"{postal_base + g % 10000:05}",
            "bucket": g,
            "trzaskowski_r1": trz_r1,
            "nawrocki_r1": naw_r1,
            "trzaskowski_r2": trz_r2,
            "nawrocki_r2": naw_r2,
        })
        polling_id += 1

df_sim = pd.DataFrame(data)


In [39]:
df_sim.head()

,teryt_gmina,polling_station_id,postal_code,bucket,trzaskowski_r1,nawrocki_r1,trzaskowski_r2,nawrocki_r2
0,140000,100000,30000,0,294,307,469,531
1,140000,100001,30000,0,300,278,517,483
2,140000,100002,30000,0,299,299,535,465
3,140000,100003,30000,0,310,278,494,506
4,140000,100004,30000,0,293,304,514,486


### IMPLEMENTACJA METOD ZAPROPONOWANYCH PRZEZ DR KONTKA

2.3. Wykrywanie wartości odstających

Główna innowacja analityczna niniejszego badania polega na oszacowaniu potencjalnego wpływu
anormalnych komisji wyborczych na poziomie ogólnokrajowym. Aby to osiągnąć, w pierwszej
kolejności zidentyfikowano wartości odstające w czterech kategoriach nieprawidłowości:

In [38]:
df = df_sim.copy()
df["postal_clean"] = df["postal_code"].astype(str).str.replace("-", "")

In [40]:
print("Liczba komisji: ", len(df))

Liczba komisji:  31627


In [41]:
cand_A = "trzaskowski"
cand_B = "nawrocki"

### 1. pop_outlier: Nadmierne poparcie dla kandydata A (względem mediany w ramach lokalnej grupy)

In [42]:
df = methods.add_anomaly_1(df, cand_A, cand_B, new_col_name="k_score_1")

In [43]:
methods.check_k_thresholds(df, cand_A, cand_B, "_k_score_1")

k > 2.0
trzaskowski: 2990
nawrocki: 3053
---
k > 2.5
trzaskowski: 1861
nawrocki: 1953
---
k > 3.0
trzaskowski: 1170
nawrocki: 1227
---


Wyniki:

Dla k=2, takich komisji, w których "za duże" poparcie ma Trzaskowski jest 2990 a Nawrocki 3053

### 2. growth_outlier: Nadmierny względny wzrost poparcia dla kandydata A między pierwszą a drugą turą, w porównaniu do odpowiedniego wzrostu poparcia dla kandydata B w tej samej grupie lokalnej;

In [44]:
df = methods.add_anomaly_2(df, cand_A, cand_B, new_col_name="k_score_2")

In [45]:
methods.check_k_thresholds(df, cand_A, cand_B, "_k_score_2")

k > 2.0
trzaskowski: 3026
nawrocki: 3098
---
k > 2.5
trzaskowski: 1915
nawrocki: 1973
---
k > 3.0
trzaskowski: 1168
nawrocki: 1271
---


### 3. Komisje, w których kand A uzyskał więcej głosów niż kand B w drugiej turze, mimo że mediana wyników w grupie wskazywała na przewagę kand B;

In [46]:
df = methods.add_anomaly_3(df, cand_A, cand_B, new_col_name="flip")

In [47]:
# na korzyść kandydata A, czyli większą medianę miał B, a więcej głosów dostał A.
cand_A, sum(df['higher_median_' + cand_B] & (df[cand_A + '_r2'] > df[cand_B + '_r2']))

('trzaskowski', 5285)

In [48]:
cand_B, sum(df['higher_median_' + cand_A] & (df[cand_B + '_r2'] > df[cand_A + '_r2']))

('nawrocki', 5079)

In [49]:
df = df.drop(
    columns=[
        f"higher_median_{cand_A}",
        f"higher_median_{cand_B}",
    ]
)

**WYNIKI**:

W grupach, w których większą medianę miał Nawrocki, było 5079 komisji, w których wyższy wynik uzyskał Trzaskowski.

W grupach, w których większą medianę miał Trzaskowski, było 5285 komisji, w których wyższy wyniki uzyskał Nawrocki.

### 4. Kandydat otrzymał mniej głosów w drugiej turze niż w pierwszej

In [50]:
cand_A, sum(df[cand_A + '_r2']<df[cand_A + '_r1'])

('trzaskowski', 0)

In [51]:
cand_B, sum(df[cand_B + '_r2']<df[cand_B + '_r1'])

('nawrocki', 0)

In [52]:
# Anomalie na korzysc
df[cand_A + '_more_votes'] = df[cand_B + '_r2']<df[cand_B + '_r1']
df[cand_B + '_more_votes'] = df[cand_A + '_r2']<df[cand_A + '_r1']

## Wyniki

In [53]:
# w zależności od wielkości k
k = 2

# deduplikacja, czyli jak wystąpiła więcej niz jedna anomalia, to liczymy taką komisję raz

### Na korzyść Nawrockiego

In [54]:
cand = "nawrocki"
opponent = "trzaskowski"
outliers_df = methods.generate_candidate_outliers(df, cand, opponent, k)

### Liczba występujących anomalii. Czyli w ilu komisjach wystąpiła np. anomalia "pop outlier"

In [55]:
bool_cols = ["pop_outlier", "growth_outlier", "flip", "more_votes"]
print(outliers_df[bool_cols].sum())

pop_outlier       3053
growth_outlier    3098
flip              5079
more_votes           0
dtype: int64


### Liczba komisji, w których wystąpiło minimum N anomalii, czyli minimum 1 (8910) albo wszystkie cztery anomalie (1)

In [56]:
true_counts = outliers_df[bool_cols].sum(axis=1)

# Convert to plain ints
count_distribution = {
    ">=1": int((true_counts >= 1).sum()),
    ">=2": int((true_counts >= 2).sum()),
    ">=3": int((true_counts >= 3).sum()),
    ">=4": int((true_counts >= 4).sum())
}

# Print row by row
for key, value in count_distribution.items():
    print(f"{key}: {value}")

>=1: 7715
>=2: 2785
>=3: 730
>=4: 0


### Próba odwzorowania tabelki z wynikami

In [57]:
# suma głosów. jeżeli w komisji występuje więcej niż jedna anomalia, sumuj jej głosy tylko raz
outliers_df["anomalies"] = outliers_df.apply(methods.assign_top_anomaly, axis=1)

In [58]:
outliers_df[cand + "_median_r2"] = outliers_df.groupby("bucket")[cand + "_r2"].transform("median")
outliers_df[opponent + "_median_r2"] = outliers_df.groupby("bucket")[opponent + "_r2"].transform("median")

In [59]:
outliers_df = methods.add_median_corrected_votes(outliers_df, cand, opponent)

In [60]:
summary_df = methods.summarize_by_anomaly(outliers_df, cand, opponent)
summary_df

,flaga,liczba,głosy nawrocki,głosy trzaskowski,różnica przed,głosy nawrocki po,głosy trzaskowski po,różnica po,zmiana
0,pop_outlier,786,419086,366914,52172,395814,390186,5628,-46544
1,growth_outlier,1850,968415,881585,86830,931453,918547,12906,-73924
2,flip,5079,2621851,2457149,164702,2513896,2565104,-51208,-215910
3,more_votes,0,0,0,0,0,0,0,0
4,łącznie,7715,4009352,3705648,303704,3841163,3873837,-32674,-336378


### Na korzyść Trzaskowskiego

In [61]:
cand = "trzaskowski"
opponent = "nawrocki"
outliers_df = methods.generate_candidate_outliers(df, cand, opponent, k)

### Liczba występujących anomalii. Czyli w ilu komisjach wystąpiła np. anomalia "pop outlier"

In [62]:
bool_cols = ["pop_outlier", "growth_outlier", "flip", "more_votes"]
print(outliers_df[bool_cols].sum())

pop_outlier       2990
growth_outlier    3026
flip              5285
more_votes           0
dtype: int64


### Liczba komisji, w których wystąpiło minimum N anomalii, czyli minimum 1 (14642) albo wszystkie cztery anomalie (0)

In [63]:
true_counts = outliers_df[bool_cols].sum(axis=1)

# Convert to plain ints
count_distribution = {
    ">=1": int((true_counts >= 1).sum()),
    ">=2": int((true_counts >= 2).sum()),
    ">=3": int((true_counts >= 3).sum()),
    ">=4": int((true_counts >= 4).sum())
}

# Print row by row
for key, value in count_distribution.items():
    print(f"{key}: {value}")

>=1: 7778
>=2: 2788
>=3: 735
>=4: 0


In [64]:
outliers_df["anomalies"] = outliers_df.apply(methods.assign_top_anomaly, axis=1)

In [65]:
outliers_df[cand + "_median_r2"] = outliers_df.groupby("bucket")[cand + "_r2"].transform("median")
outliers_df[opponent + "_median_r2"] = outliers_df.groupby("bucket")[opponent + "_r2"].transform("median")

In [66]:
outliers_df = methods.add_median_corrected_votes(outliers_df, cand, opponent)

In [67]:
summary_df = methods.summarize_by_anomaly(outliers_df, cand, opponent)
summary_df

,flaga,liczba,głosy trzaskowski,głosy nawrocki,różnica przed,głosy trzaskowski po,głosy nawrocki po,różnica po,zmiana
0,pop_outlier,710,378637,331363,47274,357563,352437,5126,-42148
1,growth_outlier,1783,931310,851690,79620,896814,886186,10628,-68992
2,flip,5285,2726555,2558445,168110,2615298,2669702,-54404,-222514
3,more_votes,0,0,0,0,0,0,0,0
4,łącznie,7778,4036502,3741498,295004,3869675,3908325,-38650,-333654
